In [1]:
import pandas as pd
import glob
import csv
import os

In [2]:
def get_description(maquina):
    df = pd.read_csv(maquina)
    return df.iloc[0]['Descrição']
        
def check_names(nome_lista, maquina_atual):
    return nome_lista in maquina_atual

def filter_name(nome_lista):
    return nome_lista.replace(";", "")

In [3]:
def listagem_maquinas():
    # Lê todos os caminhos para os arquivos das máquinas
    maquinas = glob.glob('maquinas/*.csv')
    
    # Lê a listagem de nomes das máquinas
    nomes = pd.read_csv('nomes/nomes_maquinas.csv')
    
    # Cria arquivo de saida 
    with open('lista.csv', 'w', encoding='UTF-8') as csvfile:
        filewriter = csv.writer(
            csvfile, 
            lineterminator='\n',
            delimiter=',',
            quotechar='|', 
            quoting=csv.QUOTE_MINIMAL
        )
        
        # Cria colunas
        filewriter.writerow(['Nome', 'Descricao'])

        # Percorre o nome de todas as máquinas
        for i in range(nomes.shape[0]):
            
            # Limpa o nome da máquina
            nome_puro = filter_name(nomes.iloc[i][0])

            # Procura a máquina nos arquivos individuais
            for j in range(len(maquinas)):
                
                # Caso exista, adiciona a sua descrição
                if check_names(nome_puro, maquinas[j]) == True:
                    descricao = get_description(maquinas[j])
                    filewriter.writerow([nome_puro,descricao])
                    break
                    
            # Caso não exista, adiciona informação de não exportado
            else:
                descricao = "nao exportado"
                filewriter.writerow([nome_puro,descricao])


In [11]:
def adiciona_novas_colunas():
    df = pd.read_csv('lista.csv')
    
    # remove a coluna de Descrição
    df = df.drop(['Descricao'], axis=1) 
    
    # Adiciona as seguintes colunas
    df['Patrimonio'] = 'Vazio'
    df['Modelo'] = 'Vazio'
    df['Setor'] = 'Vazio'
    df['Sala'] = 'Vazio'
    
    df.to_csv('nova_lista.csv', index=False)

In [15]:
# def adiciona_informacoes(caminho):
caminho = 'infos/Relatório de Microcomputadores agupados por UL (Endereço).csv'
count =0

def is_number(row):
    try:
        int(row[0])
        return True
    except ValueError:
        return False

cont = 0
    
# with open('teste.csv', 'w', encoding='utf-8') as csvfile:
#     filewriter = csv.writer(
#             csvfile, 
#             lineterminator='\n',
#             delimiter=',',
#             quotechar='|', 
#             quoting=csv.QUOTE_MINIMAL
#         )
        
with open(caminho, 'r', encoding='utf-8') as maquinas:
    df = pd.read_csv('nova_lista.csv')
    for row in maquinas:
        index = 0
        if 'U.A.:' in row:
            setor = row[16:-16]
            index = setor.find(' -')
            setor = setor[:index]
#                 print(setor)
        if 'Endereço:' in row:
            index = row.find('- ')
            endereco = row[index+2:-13]
            endereco = endereco.replace('SAFS QD2 LT3','')
            if '(' in endereco:
                index = endereco.find('(')
                index2 = endereco.find(')')
                setor = setor + " " + endereco[index:index2+1]
#                 print(setor)

            index = endereco.find('- ')
            index2 = endereco.find(' ', index+7)
            sala = endereco[index+2:index2]
            sala = sala.replace('SALA ','')
            sala = sala.replace(' (SALA','')
#                 sala = ""
#                 print(endereco)
#                 print(endereco)
#             print(sala)
#                 print("")


        if '3000' in row:
            index = row.find('3000')
#                 print(index)
            patrimonio = row[index:index+8]
#                 print(patrimonio)
            if not 'MODELO:' in row:
                modelo = ""
            else:
                index = row.find('MODELO:')
                index2 = row.find('-', index)
                modelo = row[index+8:index2-1]
#                 print(modelo)
            
            for i in range(df.shape[0]):
                if patrimonio in df.iloc[i]['Nome']:
                    df.at[i,'Patrimonio'] = patrimonio
                    df.at[i,'Modelo'] = modelo
                    df.at[i,'Setor'] = setor
                    df.at[i,'Sala'] = sala
                    print(df.iloc[i])
                
# df
#     print(cont)
df.to_csv('nova_lista.csv', index=False)

Nome                      HP-30001480
Patrimonio                   30001480
Modelo          ELITEDESK 800 G1 MINI
Setor          GABINETE 05 (GABINETE)
Sala                              305
Name: 59, dtype: object
Nome                      HP-30002864
Patrimonio                   30002864
Modelo          ELITEDESK 800 G1 MINI
Setor          GABINETE 05 (GABINETE)
Sala                              305
Name: 156, dtype: object
Nome                        HP-30001462
Patrimonio                     30001462
Modelo            ELITEDESK 800 G1 MINI
Setor          GABINETE 05 (ASSESSORIA)
Sala                                305
Name: 57, dtype: object
Nome                        ITN30001485
Patrimonio                     30001485
Modelo                   INFOWAY ST4265
Setor          GABINETE 05 (ASSESSORIA)
Sala                                305
Name: 372, dtype: object
Nome                         HP30001493
Patrimonio                     30001493
Modelo            ELITEDESK 800 G1 MINI
Se

Nome                        HP-30001453
Patrimonio                     30001453
Modelo            ELITEDESK 800 G1 MINI
Setor          GABINETE 09 (ASSESSORIA)
Sala                                303
Name: 54, dtype: object
Nome                        HP-30001546
Patrimonio                     30001546
Modelo            ELITEDESK 800 G1 MINI
Setor          GABINETE 09 (ASSESSORIA)
Sala                                303
Name: 63, dtype: object
Nome                         HP30001547
Patrimonio                     30001547
Modelo            ELITEDESK 800 G1 MINI
Setor          GABINETE 09 (ASSESSORIA)
Sala                                303
Name: 64, dtype: object
Nome                      HP-30001373
Patrimonio                   30001373
Modelo          ELITEDESK 800 G1 MINI
Setor          GABINETE 12 (GABINETE)
Sala                              301
Name: 38, dtype: object
Nome                       HP-30003653
Patrimonio                    30003653
Modelo        ELITEDESK 800 DM G3 MI

Nome                                           HP-30001793
Patrimonio                                        30001793
Modelo                               ELITEDESK 800 G1 MINI
Setor          CORREGEDORIA NACIONAL (GABINETE INTERNO 06)
Sala                                                   203
Name: 98, dtype: object
Nome                                           HP-30003610
Patrimonio                                        30003610
Modelo                            ELITEDESK 800 DM G3 MINI
Setor          CORREGEDORIA NACIONAL (GABINETE INTERNO 06)
Sala                                                   203
Name: 201, dtype: object
Nome                                           HP-30001795
Patrimonio                                        30001795
Modelo                               ELITEDESK 800 G1 MINI
Setor          CORREGEDORIA NACIONAL (GABINETE INTERNO 07)
Sala                                                   203
Name: 99, dtype: object
Nome                                      

Nome                       HP-30003627
Patrimonio                    30003627
Modelo        ELITEDESK 800 DM G3 MINI
Setor              ASSESSORIA JURIDICA
Sala                               206
Name: 218, dtype: object
Nome                                      HP-30002598
Patrimonio                                   30002598
Modelo                          ELITEDESK 800 G1 MINI
Setor          SECRETARIA DE ADMINISTRACAO (GABINETE)
Sala                                              T04
Name: 147, dtype: object
Nome                                PO-30001222
Patrimonio                             30001222
Modelo                      MASTER C810 MINIPRO
Setor          SECRETARIA DE GESTAO ESTRATEGICA
Sala                                        106
Name: 641, dtype: object
Nome                               ITN-30002072
Patrimonio                             30002072
Modelo                           INFOWAY ST4265
Setor          SECRETARIA DE GESTAO ESTRATEGICA
Sala                        

Nome                                           HP-30003643
Patrimonio                                        30003643
Modelo                            ELITEDESK 800 DM G3 MINI
Setor          SERVICO DE ATENDIMENTO AO USUARIO (SERVSAT)
Sala                                                  SE07
Name: 232, dtype: object
Nome                                           HP-30003644
Patrimonio                                        30003644
Modelo                            ELITEDESK 800 DM G3 MINI
Setor          SERVICO DE ATENDIMENTO AO USUARIO (SERVSAT)
Sala                                                  SE07
Name: 233, dtype: object
Nome                                           HP-30003646
Patrimonio                                        30003646
Modelo                            ELITEDESK 800 DM G3 MINI
Setor          SERVICO DE ATENDIMENTO AO USUARIO (SERVSAT)
Sala                                                  SE07
Name: 235, dtype: object
Nome                                    

...

Nome                      HP-30001893
Patrimonio                   30001893
Modelo          ELITEDESK 800 G1 MINI
Setor          SECRETARIA DISCIPLINAR
Sala                              203
Name: 111, dtype: object
Nome                      HP-30001894
Patrimonio                   30001894
Modelo          ELITEDESK 800 G1 MINI
Setor          SECRETARIA DISCIPLINAR
Sala                              203
Name: 112, dtype: object
Nome                      HP-30001899
Patrimonio                   30001899
Modelo          ELITEDESK 800 G1 MINI
Setor          SECRETARIA DISCIPLINAR
Sala                              203
Name: 113, dtype: object
Nome                     ITN-30003295
Patrimonio                   30003295
Modelo                 INFOWAY ST4265
Setor          SECRETARIA DISCIPLINAR
Sala                              203
Name: 507, dtype: object
Nome                       HP-30003579
Patrimonio                    30003579
Modelo        ELITEDESK 800 DM G3 MINI
Setor           SECRETA

Nome                                            HP-30003590
Patrimonio                                         30003590
Modelo                             ELITEDESK 800 DM G3 MINI
Setor          COMISSAO DE DEFESA DOS DIREITOS FUNDAMENTAIS
Sala                                                   CO02
Name: 182, dtype: object
Nome                                                HP-30003589
Patrimonio                                             30003589
Modelo                                 ELITEDESK 800 DM G3 MINI
Setor          COMISSAO DE DEFESA DOS DIREITOS FUNDAMENTAIS ...
Sala                                                       CO02
Name: 181, dtype: object
Nome                                            HPMINI-30003589
Patrimonio                                             30003589
Modelo                                 ELITEDESK 800 DM G3 MINI
Setor          COMISSAO DE DEFESA DOS DIREITOS FUNDAMENTAIS ...
Sala                                                       CO02
Name: 290,

Nome                          ITN-30002155
Patrimonio                        30002155
Modelo                      INFOWAY ST4265
Setor          COORDENADORIA DE ENGENHARIA
Sala                                   104
Name: 415, dtype: object
Nome                          ITN-30002158
Patrimonio                        30002158
Modelo                      INFOWAY ST4265
Setor          COORDENADORIA DE ENGENHARIA
Sala                                   104
Name: 416, dtype: object
Nome                          ITN-30002162
Patrimonio                        30002162
Modelo                      INFOWAY ST4265
Setor          COORDENADORIA DE ENGENHARIA
Sala                                   104
Name: 417, dtype: object
Nome                          ITN-30002165
Patrimonio                        30002165
Modelo                      INFOWAY ST4265
Setor          COORDENADORIA DE ENGENHARIA
Sala                                   104
Name: 418, dtype: object
Nome                          ITN-300021

Nome                                               ITN-30001131
Patrimonio                                             30001131
Modelo                                           INFOWAY ST4265
Setor          ASSESSORIA DE CERIMONIAL E EVENTOS (SALA ASSE...
Sala                                                   PLENARIO
Name: 347, dtype: object
Nome                                               ITN-30003125
Patrimonio                                             30003125
Modelo                                           INFOWAY ST4265
Setor          ASSESSORIA DE CERIMONIAL E EVENTOS (SALA ASSE...
Sala                                                   PLENARIO
Name: 491, dtype: object
Nome                                               ITN-30003131
Patrimonio                                             30003131
Modelo                                           INFOWAY ST4265
Setor          ASSESSORIA DE CERIMONIAL E EVENTOS (SALA ASSE...
Sala                                                  

Nome                                HP-30003652
Patrimonio                             30003652
Modelo                 ELITEDESK 800 DM G3 MINI
Setor          COMISSAO EXTRAORDINARIA DA SAUDE
Sala                                       CO07
Name: 241, dtype: object
Nome                                HP-30003663
Patrimonio                             30003663
Modelo                 ELITEDESK 800 DM G3 MINI
Setor          COMISSAO EXTRAORDINARIA DA SAUDE
Sala                                       CO07
Name: 251, dtype: object
Nome                                HP-30003665
Patrimonio                             30003665
Modelo                 ELITEDESK 800 DM G3 MINI
Setor          COMISSAO EXTRAORDINARIA DA SAUDE
Sala                                       CO07
Name: 252, dtype: object
Nome                                HP-30003666
Patrimonio                             30003666
Modelo                 ELITEDESK 800 DM G3 MINI
Setor          COMISSAO EXTRAORDINARIA DA SAUDE
Sala         

In [12]:
listagem_maquinas()

In [13]:
adiciona_novas_colunas()

In [20]:
df = pd.read_csv('nova_lista.csv')
for i in range(df.shape[0]):
    if '(DEPOSITO' in df.iloc[i]['Sala']:
        novo = df.iloc[i]['Sala']
        novo = novo.replace(' (DEPOSITO', '')
        df.at[i,'Sala'] = novo
df.to_csv('nova_lista.csv', index=False)

In [19]:
df = pd.read_csv('nova_lista.csv')
df.iloc[0]['Sala']

'G2 '